In [166]:
%conda install pandas numpy
%pip install ib_insync openpyxl

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 7.4 MB/s eta 0:00:00
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
from ib_insync import *
import pandas as pd


In [57]:
util.startLoop()  # necessary for notebook

ib = IB()
ib.connect()
ib.reqMarketDataType(3)

In [79]:
raw_contracts = [
    {"conId": 3691937, "exchange": "SMART"},
    {"conId": 139673266, "exchange": "SMART"},
    {"conId": 4721, "exchange": "SMART"},
    {"conId": 140070600, "exchange": "SMART"},
    {"conId": 265598, "exchange": "SMART"},
    {"conId": 5222, "exchange": "SMART"},
    {"conId": 44965120, "exchange": "SMART"},
    {"conId": 51831744, "exchange": "SMART"},
    {"conId": 9599491, "exchange": "SMART"},
    {"conId": 208813719, "exchange": "SMART"},
    {"conId": 107113386, "exchange": "SMART"},
    {"conId": 272093, "exchange": "SMART"},
    {"conId": 15124833, "exchange": "SMART"},
    {"conId": 199169591, "exchange": "SMART"},
    {"conId": 11017, "exchange": "SMART"},
    {"conId": 10291, "exchange": "SMART"},
    {"conId": 4815747, "exchange": "SMART"},
    {"conId": 76792991, "exchange": "SMART"},
    {"conId": 13977, "exchange": "SMART"},
    {"conId": 459530964, "exchange": "SMART"},
    {"conId": 365207014, "exchange": "SMART"},
]

In [80]:
contracts = [Contract(**c) for c in raw_contracts]

ib.qualifyContracts(*contracts)

index_contract = ib.qualifyContracts(Index("SPX", "CBOE"))

In [76]:
contracts


[Contract(secType='STK', conId=3691937, symbol='AMZN', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AMZN', tradingClass='NMS'),
 Contract(secType='STK', conId=139673266, symbol='AAL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAL', tradingClass='NMS'),
 Contract(secType='STK', conId=4721, symbol='AXP', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='AXP', tradingClass='AXP'),
 Contract(secType='STK', conId=140070600, symbol='AMC', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='AMC', tradingClass='AMC'),
 Contract(secType='STK', conId=265598, symbol='AAPL', exchange='SMART', primaryExchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'),
 Contract(secType='STK', conId=5222, symbol='BRK A', exchange='SMART', primaryExchange='NYSE', currency='USD', localSymbol='BRK A', tradingClass='BRK A'),
 Contract(secType='STK', conId=44965120, symbol='BKCC', exchange='SMART', p

In [83]:
GICS_MAPPINGS = {
    "Consumer, Cyclical": "Consumer Discretionary",
    "Consumer, Non-cyclical": "Consumer Staples",
    "Energy": "Energy",
    "Financial": "Financials",
    "Communications": "Information Technology",
    "Technology": "Information Technology",
}


In [81]:
contract_details = [ib.reqContractDetails(contract)[0] for contract in contracts]


In [84]:
for contract in contract_details:
    print(contract.longName, GICS_MAPPINGS[contract.industry])


AMAZON.COM INC Information Technology
AMERICAN AIRLINES GROUP INC Consumer Discretionary
AMERICAN EXPRESS CO Financials
AMC ENTERTAINMENT HLDS-CL A Consumer Discretionary
APPLE INC Information Technology
BERKSHIRE HATHAWAY INC-CL A Financials
BLACKROCK CAPITAL INVESTMENT Financials
BNP PARIBAS-ADR Financials
FORD MOTOR CO Consumer Discretionary
ALPHABET INC-CL A Information Technology
META PLATFORMS INC-CLASS A Information Technology
MICROSOFT CORP Information Technology
NETFLIX INC Information Technology
PAYPAL HOLDINGS INC Consumer Staples
PEPSICO INC Consumer Staples
NIKE INC -CL B Consumer Discretionary
NVIDIA CORP Information Technology
TESLA INC Consumer Discretionary
EXXON MOBIL CORP Energy
AIRBNB INC-CLASS A Information Technology
UBER TECHNOLOGIES INC Information Technology


Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1102, reqId -1: Connectivity between IB and Trader Workstation has been restored - data maintained. All data farms are connected: hfarm; eufarmnj; cashfarm; uscrypto; usfuture.nj; usfuture; jfarm; usfarm.nj; usfarm; euhmds; fundfarm; ushmds; secdefil.


In [103]:
index = Index("SPX", "CBOE")
ib.qualifyContracts(index)

index_data = util.df(
    ib.reqHistoricalData(
        index,
        endDateTime="",
        durationStr="5 M",
        barSizeSetting="1 day",
        whatToShow="TRADES",
        useRTH=True,
    )
)

index_data_pct_change = index_data["close"].pct_change().dropna()

In [105]:
def get_pct_change(contract):
    data = util.df(
        ib.reqHistoricalData(
            contract,
            endDateTime="",
            durationStr="5 M",
            barSizeSetting="1 day",
            whatToShow="TRADES",
            useRTH=True,
        )
    )
    return data["close"].pct_change().dropna()


def get_beta(contract):
    pct_change = get_pct_change(contract)
    return pct_change.cov(index_data_pct_change) / index_data_pct_change.var()

In [106]:
contracts_beta = [get_beta(contract) for contract in contracts]


In [129]:
pd.options.display.float_format = "{:.2f}".format

In [153]:
POSITION = 1
POSITION_SIZE = 500_000

table_1 = pd.DataFrame()
table_1["TICKER"] = [contract.symbol for contract in contracts]
table_1["GICS"] = [GICS_MAPPINGS[contract.industry] for contract in contract_details]
table_1["BETA (5MONTH)"] = contracts_beta
table_1["POSITION"] = POSITION
table_1["NET EXPOSURE"] = POSITION_SIZE
table_1["BETA-RISK"] = (
    table_1["BETA (5MONTH)"] * table_1["NET EXPOSURE"] * table_1["POSITION"]
)

# add a total row at the bottom, with TOTAL being in the TICKER column
table_1 = table_1.append(table_1.sum(numeric_only=True), ignore_index=True).fillna(
    "TOTAL"
)
table_1.loc[21, ["GICS", "BETA (5MONTH)"]] = ""

table_1

/var/folders/bj/1ksjxyv53h39fz6kk0jc_3xm0000gn/T/ipykernel_4911/2144874372.py:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_1 = table_1.append(table_1.sum(numeric_only=True), ignore_index=True).fillna('TOTAL')


,TICKER,GICS,BETA (5MONTH),POSITION,NET EXPOSURE,BETA-RISK
0,AMZN,Information Technology,1.76,1.00,500000.00,877550.24
1,AAL,Consumer Discretionary,1.22,1.00,500000.00,609247.72
2,AXP,Financials,1.04,1.00,500000.00,518989.83
3,AMC,Consumer Discretionary,2.17,1.00,500000.00,1083877.85
4,AAPL,Information Technology,1.37,1.00,500000.00,683928.76
5,BRK A,Financials,0.81,1.00,500000.00,407227.39
6,BKCC,Financials,0.82,1.00,500000.00,407901.65
7,BNPQY,Financials,0.97,1.00,500000.00,485183.03
8,F,Consumer Discretionary,1.52,1.00,500000.00,759992.40
9,GOOGL,Information Technology,1.47,1.00,500000.00,732958.93


In [164]:
import locale

locale.setlocale(locale.LC_ALL, "en_US.UTF-8")

table_2 = table_1.copy().groupby("GICS").sum(numeric_only=True)
table_2.drop("", inplace=True)
table_2.reset_index(inplace=True)
table_2.rename(
    columns={"GICS": "GICS SECTOR ALLOCATION", "NET EXPOSURE": "NET CASH"}, inplace=True
)
table_2 = table_2.append(table_2.sum(numeric_only=True), ignore_index=True).fillna(
    "TOTAL"
)
table_2["NET CASH"] = table_2["NET CASH"].apply(
    lambda x: locale.currency(x, grouping=True)
)
table_2["BETA-RISK"] = table_2["BETA-RISK"].apply(
    lambda x: locale.currency(x, grouping=True)
)
table_2

/var/folders/bj/1ksjxyv53h39fz6kk0jc_3xm0000gn/T/ipykernel_4911/2733000249.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  table_2 = table_2.append(table_2.sum(numeric_only=True), ignore_index=True).fillna('TOTAL')


,GICS SECTOR ALLOCATION,POSITION,NET CASH,BETA-RISK
0,Consumer Discretionary,5.00,"$2,500,000.00","$4,058,664.48"
1,Consumer Staples,2.00,"$1,000,000.00","$813,926.53"
2,Energy,1.00,"$500,000.00","$362,659.30"
3,Financials,4.00,"$2,000,000.00","$1,819,301.89"
4,Information Technology,9.00,"$4,500,000.00","$7,292,999.18"
5,TOTAL,21.00,"$10,500,000.00","$14,347,551.38"


In [7]:
GICS_TO_HEDGE = {
    "Information Technology": "CIBR",
    "Consumer Discretionary": "PAWZ",
    "Financials": "IAE",
    "Consumer Staples": "VDC",
    "Energy": "TAN",
}

In [13]:
table_3 = pd.DataFrame()
table_3["GICS SECTOR HEDGE"] = table_2["GICS SECTOR ALLOCATION"]
table_3["TICKER"] = table_3["GICS SECTOR HEDGE"].map(GICS_TO_HEDGE)
table_3["CASH ALLOCATION"] = table_2["BETA-RISK"]
table_3.loc[5, "TICKER"] = ""

table_3

,GICS SECTOR HEDGE,TICKER,CASH ALLOCATION
0,Consumer Discretionary,PAWZ,"$4,058,664.48"
1,Consumer Staples,VDC,"$813,926.53"
2,Energy,TAN,"$362,659.30"
3,Financials,IAE,"$1,819,301.89"
4,Information Technology,CIBR,"$7,292,999.18"
5,TOTAL,,"$14,347,551.38"


In [15]:
import os

if not os.path.exists("outputs"):
    os.mkdir("outputs")

with pd.ExcelWriter(f"./outputs/Portfolio Construction Theo POMIES.xlsx") as writer:
    table_1.to_excel(writer, sheet_name="Table 1", index=False)
    table_2.to_excel(writer, sheet_name="Table 2", index=False)
    table_3.to_excel(writer, sheet_name="Table 3", index=False)
